In [1]:
from liveF1Wrapper.adapter import LivetimingF1Adapter, LivetimingF1Request, LivetimingF1GetData
from liveF1Wrapper.season import Season
from liveF1Wrapper.functions import *
from liveF1Wrapper.events import *
import urllib
import json
import numpy as np

In [3]:
adapter = LivetimingF1Adapter()
endpoint = "https://livetiming.formula1.com/static/2024/2024-09-01_Italian_Grand_Prix/2024-09-01_Race/SessionData.jsonStream"
res_text = adapter.get(endpoint=endpoint)
records = res_text.split('\r\n')[:-1]

tl = 12
data = dict((r[:12], json.loads(r[12:])) for r in records)
data

{'00:13:12.826': {'Series': {'0': {'Utc': '2024-09-01T12:07:46.299Z',
    'Lap': 1}}},
 '00:55:50.494': {'StatusSeries': {'1': {'Utc': '2024-09-01T13:03:34.412Z',
    'SessionStatus': 'Started'}}},
 '00:57:18.493': {'Series': {'1': {'Utc': '2024-09-01T13:05:02.411Z',
    'Lap': 2}}},
 '00:58:43.480': {'Series': {'2': {'Utc': '2024-09-01T13:06:27.398Z',
    'Lap': 3}}},
 '01:00:08.733': {'Series': {'3': {'Utc': '2024-09-01T13:07:52.651Z',
    'Lap': 4}}},
 '01:01:33.428': {'Series': {'4': {'Utc': '2024-09-01T13:09:17.346Z',
    'Lap': 5}}},
 '01:02:58.083': {'Series': {'5': {'Utc': '2024-09-01T13:10:42.001Z',
    'Lap': 6}}},
 '01:04:22.700': {'Series': {'6': {'Utc': '2024-09-01T13:12:06.618Z',
    'Lap': 7}}},
 '01:05:47.129': {'Series': {'7': {'Utc': '2024-09-01T13:13:31.047Z',
    'Lap': 8}}},
 '01:07:11.690': {'Series': {'8': {'Utc': '2024-09-01T13:14:55.608Z',
    'Lap': 9}}},
 '01:08:36.065': {'Series': {'9': {'Utc': '2024-09-01T13:16:19.983Z',
    'Lap': 10}}},
 '01:10:00.163': {

In [4]:
endpoint = "https://livetiming.formula1.com/static/2024/2024-09-01_Italian_Grand_Prix/2024-09-01_Race/SessionData.jsonStream"
data = LivetimingF1GetData(endpoint, stream=True)
data

{'00:13:12.826': {'Series': {'0': {'Utc': '2024-09-01T12:07:46.299Z',
    'Lap': 1}}},
 '00:55:50.494': {'StatusSeries': {'1': {'Utc': '2024-09-01T13:03:34.412Z',
    'SessionStatus': 'Started'}}},
 '00:57:18.493': {'Series': {'1': {'Utc': '2024-09-01T13:05:02.411Z',
    'Lap': 2}}},
 '00:58:43.480': {'Series': {'2': {'Utc': '2024-09-01T13:06:27.398Z',
    'Lap': 3}}},
 '01:00:08.733': {'Series': {'3': {'Utc': '2024-09-01T13:07:52.651Z',
    'Lap': 4}}},
 '01:01:33.428': {'Series': {'4': {'Utc': '2024-09-01T13:09:17.346Z',
    'Lap': 5}}},
 '01:02:58.083': {'Series': {'5': {'Utc': '2024-09-01T13:10:42.001Z',
    'Lap': 6}}},
 '01:04:22.700': {'Series': {'6': {'Utc': '2024-09-01T13:12:06.618Z',
    'Lap': 7}}},
 '01:05:47.129': {'Series': {'7': {'Utc': '2024-09-01T13:13:31.047Z',
    'Lap': 8}}},
 '01:07:11.690': {'Series': {'8': {'Utc': '2024-09-01T13:14:55.608Z',
    'Lap': 9}}},
 '01:08:36.065': {'Series': {'9': {'Utc': '2024-09-01T13:16:19.983Z',
    'Lap': 10}}},
 '01:10:00.163': {

In [5]:
# [json.loads(val) for val in list(data.values())]

for val in list(data.values()):
    print(val)
    # json.loads(val)

{'Series': {'0': {'Utc': '2024-09-01T12:07:46.299Z', 'Lap': 1}}}
{'StatusSeries': {'1': {'Utc': '2024-09-01T13:03:34.412Z', 'SessionStatus': 'Started'}}}
{'Series': {'1': {'Utc': '2024-09-01T13:05:02.411Z', 'Lap': 2}}}
{'Series': {'2': {'Utc': '2024-09-01T13:06:27.398Z', 'Lap': 3}}}
{'Series': {'3': {'Utc': '2024-09-01T13:07:52.651Z', 'Lap': 4}}}
{'Series': {'4': {'Utc': '2024-09-01T13:09:17.346Z', 'Lap': 5}}}
{'Series': {'5': {'Utc': '2024-09-01T13:10:42.001Z', 'Lap': 6}}}
{'Series': {'6': {'Utc': '2024-09-01T13:12:06.618Z', 'Lap': 7}}}
{'Series': {'7': {'Utc': '2024-09-01T13:13:31.047Z', 'Lap': 8}}}
{'Series': {'8': {'Utc': '2024-09-01T13:14:55.608Z', 'Lap': 9}}}
{'Series': {'9': {'Utc': '2024-09-01T13:16:19.983Z', 'Lap': 10}}}
{'Series': {'10': {'Utc': '2024-09-01T13:17:44.081Z', 'Lap': 11}}}
{'Series': {'11': {'Utc': '2024-09-01T13:19:08.327Z', 'Lap': 12}}}
{'Series': {'12': {'Utc': '2024-09-01T13:20:33.062Z', 'Lap': 13}}}
{'Series': {'13': {'Utc': '2024-09-01T13:21:57.706Z', 'Lap'

In [17]:
records = []

endpoint = "https://livetiming.formula1.com/static/2020/2020-07-05_Austrian_Grand_Prix/2020-07-05_Race/SessionData.jsonStream"
# endpoint = "https://livetiming.formula1.com/static/2020/2020-07-05_Austrian_Grand_Prix/2020-07-03_Practice_1/SessionData.jsonStream"
data = LivetimingF1GetData(endpoint, stream=True)
data

records = []
list(list(data.values())[0].values())[0]

for key, value in data.items():
    for driver_no, info in value.items():
        try:
            record = {
                **list(info.values())[0]
            }
            records.append(record)
        except:
            pass
        
pd.DataFrame(records)

,Utc,Lap,SessionStatus,TrackStatus
0,2020-07-05T12:40:04.65Z,1.0,NaN,NaN
1,2020-07-05T13:12:47.228Z,NaN,Started,NaN
2,2020-07-05T13:13:58.983Z,2.0,NaN,NaN
3,2020-07-05T13:15:08.42Z,3.0,NaN,NaN
4,2020-07-05T13:16:18.019Z,4.0,NaN,NaN
...,...,...,...,...
83,2020-07-05T14:42:12.361Z,NaN,NaN,AllClear
84,2020-07-05T14:42:34.813Z,71.0,NaN,NaN
85,2020-07-05T14:43:43.214Z,NaN,Finished,NaN
86,2020-07-05T14:47:24.24Z,NaN,Finalised,NaN


In [18]:
data

{'00:07:56.420': {'Series': {'0': {'Utc': '2020-07-05T12:40:04.65Z',
    'Lap': 1}}},
 '00:32:47.006': {'StatusSeries': {'1': {'Utc': '2020-07-05T13:12:47.228Z',
    'SessionStatus': 'Started'}}},
 '00:33:58.761': {'Series': {'1': {'Utc': '2020-07-05T13:13:58.983Z',
    'Lap': 2}}},
 '00:35:08.198': {'Series': {'2': {'Utc': '2020-07-05T13:15:08.42Z',
    'Lap': 3}}},
 '00:36:17.797': {'Series': {'3': {'Utc': '2020-07-05T13:16:18.019Z',
    'Lap': 4}}},
 '00:37:27.365': {'Series': {'4': {'Utc': '2020-07-05T13:17:27.587Z',
    'Lap': 5}}},
 '00:38:36.531': {'Series': {'5': {'Utc': '2020-07-05T13:18:36.753Z',
    'Lap': 6}}},
 '00:39:45.957': {'Series': {'6': {'Utc': '2020-07-05T13:19:46.179Z',
    'Lap': 7}}},
 '00:40:54.966': {'Series': {'7': {'Utc': '2020-07-05T13:20:55.188Z',
    'Lap': 8}}},
 '00:42:04.149': {'Series': {'8': {'Utc': '2020-07-05T13:22:04.371Z',
    'Lap': 9}}},
 '00:43:13.521': {'Series': {'9': {'Utc': '2020-07-05T13:23:13.743Z',
    'Lap': 10}}},
 '00:49:57.794': {'S

In [8]:
list(list(data.values())[0].values())[0]
for key, value in data.items():
    for driver_no, info in value.items():
        print(info)

[]
[{'Utc': '2020-07-03T08:28:18.593Z', 'TrackStatus': 'AllClear'}]
{'1': {'Utc': '2020-07-03T09:00:00.119Z', 'SessionStatus': 'Started'}}
{'2': {'Utc': '2020-07-03T10:10:44.359Z', 'TrackStatus': 'VSCDeployed'}}
{'3': {'Utc': '2020-07-03T10:11:19.058Z', 'TrackStatus': 'VSCEnding'}}
{'4': {'Utc': '2020-07-03T10:11:32.033Z', 'TrackStatus': 'AllClear'}}
{'5': {'Utc': '2020-07-03T10:14:14.877Z', 'TrackStatus': 'Yellow'}}
{'6': {'Utc': '2020-07-03T10:14:25.646Z', 'TrackStatus': 'AllClear'}}
{'7': {'Utc': '2020-07-03T10:19:26.39Z', 'TrackStatus': 'Yellow'}}
{'8': {'Utc': '2020-07-03T10:19:35.374Z', 'TrackStatus': 'AllClear'}}
{'41': {'Utc': '2020-07-03T09:00:00.119Z', 'SessionStatus': 'Started'}}


In [2]:
def test(path): 
    records = []

    endpoint = path + "SessionData.jsonStream"
    data = LivetimingF1GetData(endpoint, stream=True)
    data

    records = []
    list(list(data.values())[0].values())[0]

    for key, value in data.items():
        for driver_no, info in value.items():
            record = {
                **list(info.values())[0]
            }
            records.append(record)
    return pd.DataFrame(records)

In [4]:
seasons = [
    "2020",
    "2021",
    "2022",
    "2023",
    "2024"
]

season = get_season("2024")

for year in seasons[0:2]:
    print(year)
    try:
        season = get_season(year)
        for meeting in season.meetings[0:2]:
            print(meeting.officialname)
            for session in meeting.sessions:
                try:
                    if hasattr(session,"full_path"):
                        print(session.full_path)
                        display(test(session.full_path))
                except Exception as e:
                    print(e)
                    pass
    except:
        pass

2020
FORMULA 1 ROLEX GROSSER PREIS VON ÖSTERREICH 2020
https://livetiming.formula1.com/static/2020/2020-07-05_Austrian_Grand_Prix/2020-07-03_Practice_1/
'list' object has no attribute 'values'
https://livetiming.formula1.com/static/2020/2020-07-05_Austrian_Grand_Prix/2020-07-03_Practice_2/
'list' object has no attribute 'values'
https://livetiming.formula1.com/static/2020/2020-07-05_Austrian_Grand_Prix/2020-07-04_Practice_3/
'list' object has no attribute 'values'
https://livetiming.formula1.com/static/2020/2020-07-05_Austrian_Grand_Prix/2020-07-04_Qualifying/
'list' object has no attribute 'values'
https://livetiming.formula1.com/static/2020/2020-07-05_Austrian_Grand_Prix/2020-07-05_Race/


,Utc,Lap,SessionStatus,TrackStatus
0,2020-07-05T12:40:04.65Z,1.0,NaN,NaN
1,2020-07-05T13:12:47.228Z,NaN,Started,NaN
2,2020-07-05T13:13:58.983Z,2.0,NaN,NaN
3,2020-07-05T13:15:08.42Z,3.0,NaN,NaN
4,2020-07-05T13:16:18.019Z,4.0,NaN,NaN
...,...,...,...,...
83,2020-07-05T14:42:12.361Z,NaN,NaN,AllClear
84,2020-07-05T14:42:34.813Z,71.0,NaN,NaN
85,2020-07-05T14:43:43.214Z,NaN,Finished,NaN
86,2020-07-05T14:47:24.24Z,NaN,Finalised,NaN


https://livetiming.formula1.com/static/2020/2020-07-05_Austrian_Grand_Prix/2020-07-03_Practice_1/
'list' object has no attribute 'values'
https://livetiming.formula1.com/static/2020/2020-07-05_Austrian_Grand_Prix/2020-07-03_Practice_2/
'list' object has no attribute 'values'
https://livetiming.formula1.com/static/2020/2020-07-05_Austrian_Grand_Prix/2020-07-04_Practice_3/
'list' object has no attribute 'values'
https://livetiming.formula1.com/static/2020/2020-07-05_Austrian_Grand_Prix/2020-07-04_Qualifying/
'list' object has no attribute 'values'
https://livetiming.formula1.com/static/2020/2020-07-05_Austrian_Grand_Prix/2020-07-05_Race/


,Utc,Lap,SessionStatus,TrackStatus
0,2020-07-05T12:40:04.65Z,1.0,NaN,NaN
1,2020-07-05T13:12:47.228Z,NaN,Started,NaN
2,2020-07-05T13:13:58.983Z,2.0,NaN,NaN
3,2020-07-05T13:15:08.42Z,3.0,NaN,NaN
4,2020-07-05T13:16:18.019Z,4.0,NaN,NaN
...,...,...,...,...
83,2020-07-05T14:42:12.361Z,NaN,NaN,AllClear
84,2020-07-05T14:42:34.813Z,71.0,NaN,NaN
85,2020-07-05T14:43:43.214Z,NaN,Finished,NaN
86,2020-07-05T14:47:24.24Z,NaN,Finalised,NaN


FORMULA 1 PIRELLI GROSSER PREIS DER STEIERMARK 2020
https://livetiming.formula1.com/static/2020/2020-07-12_Styrian_Grand_Prix/2020-07-10_Practice_1/
'list' object has no attribute 'values'
https://livetiming.formula1.com/static/2020/2020-07-12_Styrian_Grand_Prix/2020-07-10_Practice_2/


...

https://livetiming.formula1.com/static/2020/2020-07-12_Styrian_Grand_Prix/2020-07-10_Practice_1/
'list' object has no attribute 'values'
https://livetiming.formula1.com/static/2020/2020-07-12_Styrian_Grand_Prix/2020-07-10_Practice_2/


,Utc,SessionStatus,TrackStatus
0,2020-07-10T13:05:00.289Z,Started,NaN
1,2020-07-10T13:17:49.349Z,NaN,Yellow
2,2020-07-10T13:17:59.955Z,Aborted,NaN
3,2020-07-10T13:18:00.048Z,NaN,Red
4,2020-07-10T13:28:23.818Z,Inactive,NaN
5,2020-07-10T13:29:00.94Z,NaN,AllClear
6,2020-07-10T13:30:00.284Z,Started,NaN
7,2020-07-10T13:38:30.276Z,NaN,Yellow
8,2020-07-10T13:38:47.676Z,NaN,AllClear
9,2020-07-10T14:19:14.115Z,NaN,Yellow


https://livetiming.formula1.com/static/2020/2020-07-12_Styrian_Grand_Prix/2020-07-11_Practice_3/
'list' object has no attribute 'values'
https://livetiming.formula1.com/static/2020/2020-07-12_Styrian_Grand_Prix/2020-07-11_Qualifying/
'list' object has no attribute 'values'
https://livetiming.formula1.com/static/2020/2020-07-12_Styrian_Grand_Prix/2020-07-12_Race/


,Utc,Lap,SessionStatus,TrackStatus
0,2020-07-12T12:40:05.118Z,1.0,NaN,NaN
1,2020-07-12T13:12:50.825Z,NaN,Started,NaN
2,2020-07-12T13:13:53.959Z,NaN,NaN,SCDeployed
3,2020-07-12T13:14:09.293Z,2.0,NaN,NaN
4,2020-07-12T13:16:14.786Z,3.0,NaN,NaN
...,...,...,...,...
74,2020-07-12T14:33:25.306Z,70.0,NaN,NaN
75,2020-07-12T14:34:33.486Z,71.0,NaN,NaN
76,2020-07-12T14:35:41.753Z,NaN,Finished,NaN
77,2020-07-12T14:39:38.755Z,NaN,Finalised,NaN


2021
FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2021
https://livetiming.formula1.com/static/2021/2021-03-28_Bahrain_Grand_Prix/2021-03-26_Practice_1/
'list' object has no attribute 'values'
https://livetiming.formula1.com/static/2021/2021-03-28_Bahrain_Grand_Prix/2021-03-26_Practice_2/
'list' object has no attribute 'values'
https://livetiming.formula1.com/static/2021/2021-03-28_Bahrain_Grand_Prix/2021-03-27_Practice_3/
'list' object has no attribute 'values'
https://livetiming.formula1.com/static/2021/2021-03-28_Bahrain_Grand_Prix/2021-03-27_Qualifying/
'list' object has no attribute 'values'
https://livetiming.formula1.com/static/2021/2021-03-28_Bahrain_Grand_Prix/2021-03-28_Race/
'list' object has no attribute 'values'
https://livetiming.formula1.com/static/2021/2021-03-28_Bahrain_Grand_Prix/2021-03-26_Practice_1/
'list' object has no attribute 'values'
https://livetiming.formula1.com/static/2021/2021-03-28_Bahrain_Grand_Prix/2021-03-26_Practice_2/
'list' object has no attribute 'values'